#### Final Season Rankings

In [42]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [31]:
awards = pd.read_csv("project_data/awards_players.csv")
coaches = pd.read_csv(Path("project_data", "coaches.csv"))
players_teams = pd.read_csv(Path("project_data", "players_teams.csv"))
players = pd.read_csv(Path("project_data", "players.csv"))
series_post = pd.read_csv(Path("project_data", "series_post.csv"))
teams_post = pd.read_csv(Path("project_data", "teams_post.csv"))
teams = pd.read_csv(Path("project_data", "teams.csv"))

In [40]:

# Choose the year to analyze
year_to_analyze = 4 # change this as needed

# Filter for that year
df_year = players_teams[players_teams["year"] == year_to_analyze].copy()
# df_year = df_year[(df_year["GP"] >= 10) & (df_year["minutes"] >= 100)]

# Apply Game Score formula
base_score = (
    df_year["points"]
    + 0.4 * df_year["fgMade"]
    - 0.7 * df_year["fgAttempted"]
    - 0.4 * (df_year["ftAttempted"] - df_year["ftMade"])
    + 0.7 * df_year["oRebounds"]
    + 0.3 * df_year["dRebounds"]
    + df_year["steals"]
    + 0.7 * df_year["assists"]
    + 0.7 * df_year["blocks"]
    - 0.4 * df_year["PF"]
    - df_year["turnovers"]
)

df_year["Game_Score_Total"] = base_score
df_year["Game_Score_Per_Game"] = base_score / df_year["GP"].replace(0, np.nan)
df_year["Game_Score_Per_Minute"] = base_score / df_year["minutes"].replace(0, np.nan)

# Sort by Game Score (descending)
df_sorted_total = df_year.sort_values("Game_Score_Total", ascending=False)
df_sorted_game = df_year.sort_values("Game_Score_Per_Game", ascending=False)
df_sorted_minute = df_year.sort_values("Game_Score_Per_Minute", ascending=False)

# Save to CSV
output_file = f"game_scores_{year_to_analyze}.csv"
# df_sorted.to_csv(output_file, index=False)

print(f"✅ Game scores calculated and saved to {output_file}")
print(df_sorted_total[["playerID", "tmID", "year", "Game_Score_Total"]].head(8), "\n")

print(df_sorted_game[["playerID", "tmID", "year", "Game_Score_Per_Game"]].head(8),"\n")

print(df_sorted_minute[["playerID", "tmID", "year", "Game_Score_Per_Minute"]].head(8), "\n")


✅ Game scores calculated and saved to game_scores_4.csv
        playerID tmID  year  Game_Score_Total
732   jacksla01w  SEA     4             558.3
279   catchta01w  IND     4             529.7
681   holdsch01w  WAS     4             415.8
1478  smithka01w  MIN     4             411.5
1578  swoopsh01w  HOU     4             388.0
575   griffyo01w  SAC     4             386.5
263    cashsw01w  DET     4             385.7
1601  teaslni01w  LAS     4             358.2 

        playerID tmID  year  Game_Score_Per_Game
732   jacksla01w  SEA     4            16.918182
279   catchta01w  IND     4            15.579412
681   holdsch01w  WAS     4            15.400000
906   leslili01w  LAS     4            13.621739
1578  swoopsh01w  HOU     4            12.516129
322   coopecy01w  HOU     4            12.225000
1478  smithka01w  MIN     4            12.102941
263    cashsw01w  DET     4            11.687879 

        playerID tmID  year  Game_Score_Per_Minute
732   jacksla01w  SEA     4       

In [41]:
# --- Weighted team average section ---
# Weighted average of Game_Score_Per_Minute, weighted by Game_Score_Total
def weighted_avg(group):
    total_weight = group["Game_Score_Total"].sum()
    if total_weight == 0:
        return np.nan
    weighted_mean = (group["Game_Score_Per_Minute"] * group["Game_Score_Total"]).sum() / total_weight
    return weighted_mean

# Calculate team-level metrics
team_avg = df_year.groupby("tmID").agg({
    "Game_Score_Total": "mean",
    "Game_Score_Per_Game": "mean",
    "Game_Score_Per_Minute": "mean"
}).rename(columns={
    "Game_Score_Total": "Team_Avg_Score_Total",
    "Game_Score_Per_Game": "Team_Avg_Score_Per_Game",
    "Game_Score_Per_Minute": "Team_Avg_Score_Per_Minute"
}).reset_index()

# Add weighted average separately
team_avg["Team_Weighted_Avg_Per_Minute"] = df_year.groupby("tmID").apply(weighted_avg).values

# --- Add conference info ---
teams_year = teams[teams["year"] == year_to_analyze][["tmID", "confID", "name", "rank", "won", "lost"]]
teams_year.rename(columns={"rank": "Actual_Rank"}, inplace=True)

team_avg = pd.merge(team_avg, teams_year, on="tmID", how="left")

# --- Predict rankings per conference ---
team_avg["Predicted_Rank_Conf"] = (
    team_avg.groupby("confID")["Team_Weighted_Avg_Per_Minute"]
    .rank(ascending=False, method="first")
)

# --- Compare predicted vs actual per conference ---
comparison = team_avg[["confID", "tmID", "name", "Team_Weighted_Avg_Per_Minute",
                       "Predicted_Rank_Conf", "Actual_Rank", "won", "lost"]].copy()
comparison["Rank_Diff"] = comparison["Predicted_Rank_Conf"] - comparison["Actual_Rank"]

# --- Display results by conference ---
for conf in comparison["confID"].dropna().unique():
    print(f"\n🏆 Conference: {conf}")
    conf_df = (
        comparison[comparison["confID"] == conf]
        .sort_values("Predicted_Rank_Conf")
        .reset_index(drop=True)
    )
    print(conf_df[["tmID", "name", "Predicted_Rank_Conf", "Actual_Rank", "Rank_Diff", "won", "lost"]])


🏆 Conference: EA
  tmID                name  Predicted_Rank_Conf  Actual_Rank  Rank_Diff  won  \
0  IND       Indiana Fever                  1.0            5       -4.0   16   
1  DET       Detroit Shock                  2.0            1        1.0   25   
2  WAS  Washington Mystics                  3.0            7       -4.0    9   
3  CON     Connecticut Sun                  4.0            3        1.0   18   
4  NYL    New York Liberty                  5.0            6       -1.0   16   
5  CLE   Cleveland Rockers                  6.0            4        2.0   17   
6  CHA     Charlotte Sting                  7.0            2        5.0   18   

   lost  
0    18  
1     9  
2    25  
3    16  
4    18  
5    17  
6    16  

🏆 Conference: WE
  tmID                      name  Predicted_Rank_Conf  Actual_Rank  Rank_Diff  \
0  SEA             Seattle Storm                  1.0            5       -4.0   
1  LAS        Los Angeles Sparks                  2.0            1        1.0   


C:\Users\Utilizador\AppData\Local\Temp\ipykernel_11804\1829586143.py:22: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  team_avg["Team_Weighted_Avg_Per_Minute"] = df_year.groupby("tmID").apply(weighted_avg).values
